In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#MAKES IT SO ALL COLUMNS DISPLAY
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv')

FileNotFoundError: [Errno 2] File b'nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv' does not exist: b'nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv'

In [120]:
df_unedited = df.copy()

In [121]:
np.sort(df['HomeTeam'].unique())

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAC', 'JAX', 'KC', 'LA', 'LAC',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SD',
       'SEA', 'SF', 'STL', 'TB', 'TEN', 'WAS'], dtype=object)

In [122]:
divisions = {
    'GB' : 'NFC North',
    'CHI' : 'NFC North',
    'MIN' : 'NFC North',
    'DET' : 'NFC North',
    'PHI' : 'NFC East',
    'NYG' : 'NFC East', 
    'DAL' : 'NFC East', 
    'WAS' : 'NFC East',
    'NO' : 'NFC South',
    'CAR' : 'NFC South', 
    'TB' : 'NFC South', 
    'ATL' : 'NFC South',
    'SF' : 'NFC West',
    'LA' : 'NFC West', 
    'STL' : 'NFC West', 
    'ARI' : 'NFC West', 
    'SEA' : 'NFC West',
    'BAL' : 'AFC North', 
    'PIT' : 'AFC North', 
    'CLE' : 'AFC North', 
    'CIN' : 'AFC North',
    'NE' : 'AFC East',
    'MIA' : 'AFC East', 
    'BUF' : 'AFC East', 
    'NYJ' : 'AFC East',
    'JAX' : 'AFC South', 
    'JAC' : 'AFC South', 
    'HOU' : 'AFC South', 
    'TEN' : 'AFC South', 
    'IND' : 'AFC South',
    'SD' : 'AFC West', 
    'LAC' : 'AFC West', 
    'OAK' : 'AFC West',
    'DEN': 'AFC West',
    'KC' : 'AFC West'
}

In [123]:
adv_stats_dict = {
    'ExpPts' : ('The value of the current down, distance, and field position situation in terms of future' + 
                'expected net point advantage. In other words, it is the net point value a team can expect' + 
                'given a particular combination of down, distance, and field position. First and goal' + 
                'at the one represents an EP near 6, while 3rd and 20 at a team\'s own one yard line' + 
                'represents an EP of about -2. EP differs from Win Probability (WP) in that it does' + 
                'not take into account the game score and time remaining.'),
    'EPA' : ('The difference between the Expected Points (EP) at the start of a play and the EP' + 
             'at the end of they play. EPA is the measure of a play’s impact on the score of the game.' + 
             ' An individual player’s EPA is the sum of the EPA of the plays in which that player was' + 
             'directly involved. Being directly involved is defined as an offensive player who ran,' + 
             ' threw, or kicked the ball, was targeted by a pass, or flagged for a penalty.'),
    'Home_WP_pre' : ('The probability that a team will win a game in progress, given a particular' + 
                  ' combination of circumstances including score, time remaining, field position,' + 
                  ' down, and to go distance. WP is based on a model built on actual outcomes of NFL games' + 
                  'from recent seasons that featured similar circumstances.'),
    'WPA' : ("The difference between a team’s Win Probability (WP) at the start of a play" + 
             " and the WP at the end of the play. WPA is the measure of a play’s impact on" + 
             " the outcome of a game. An individual player’s WPA is the sum of the WPA of" +  
             " the plays in which that player was directly involved. Being directly involved" + 
             " is defined as an offensive player who ran, threw, or kicked the ball, was targeted" + 
             " by a pass, or flagged for a penalty. Defensive players are credited for WPA when they" +  
             " tackle or sack the ball carrier, are credited with an assisted tackle or sack, cause a fumble," +
             " defend a pass, or are flagged for a penalty.")
}

In [124]:
#df = df.drop(['Date', 'yrdln', 'time', 'TimeUnder', 
#              'PlayTimeDiff', 'PlayAttempted', 'sp', 
#              'Interceptor', 'Rusher', 'Rusher_ID', 'Receiver', 
#              'Receiver_ID', 'Returner', 'BlockingPlayer', 'Tackler1', 
#              'Tackler2', 'RecFumbPlayer', 'Challenge.Replay', 'ChalReplayResult', 
#              'PenalizedPlayer', 'ScoreDiff', 'AbsScoreDiff', 'airEPA', 'yacEPA', 'Away_WP_pre', 
#              'Away_WP_post', 'airWPA', 'yacWPA', 'Timeout_Indicator', 'Timeout_Team', 
#              'posteam_timeouts_pre', 'HomeTimeouts_Remaining_Pre', 'AwayTimeouts_Remaining_Pre', 
#              'HomeTimeouts_Remaining_Post', 'AwayTimeouts_Remaining_Post', 'Win_Prob', 
#              'No_Score_Prob', 'Opp_Field_Goal_Prob', 'Opp_Safety_Prob', 'Opp_Touchdown_Prob', 
#              'Field_Goal_Prob', 'Safety_Prob', 'Touchdown_Prob', 'ExPoint_Prob', 'TwoPoint_Prob'], axis = 1)

In [125]:
#GAMES THAT ACTUALLY HAVE AN ENDGAME ENTRY FOR WINNER/LOSER
games_with_endgame = set(df[df['desc'] == 'END GAME']['GameID'])

In [126]:
game_ids = list(df['GameID'].unique())

In [127]:
def home_score(row):
    if row['posteam'] == row['HomeTeam']:
        return row['PosTeamScore']
    else:
        return row['DefTeamScore']
def away_score(row):
    if row['posteam'] == row['AwayTeam']:
        return row['PosTeamScore']
    else:
        return row['DefTeamScore']
def make_divisions(row):
    if divisions[row['HomeTeam']] == divisions[row['AwayTeam']]:
        return True
    else:
        return False
def make_conferences(row):
    if divisions[row['HomeTeam']][0:3] == divisions[row['AwayTeam']][0:3]:
        return True
    else:
        return False

In [128]:
df['HomeScore'] = df.apply(lambda x : home_score(x), axis = 1)

In [129]:
df['AwayScore'] = df.apply(lambda x : away_score(x), axis = 1)

In [130]:
df['Divisional'] = df.apply(lambda x : make_divisions(x), axis = 1)

In [131]:
df['Conference'] = df.apply(lambda x : make_conferences(x), axis = 1)

In [132]:
#FIXES END OF GAME ISSUES
df['HomeScore'] = df['HomeScore'].fillna(method = 'ffill')
df['AwayScore'] = df['AwayScore'].fillna(method = 'ffill')

In [133]:
#GAMES THAT END ON A PLAY AS OPPOSED TO HAVING AN ENDGAME ENTRY
games_end_on_play = list(df[[df['GameID'][x] not in games_with_endgame for x in range(len(df))]]['GameID'].unique())

In [134]:
games_end_on_play_df = df[[df['GameID'][x] in games_end_on_play for x in range(len(df))]]

In [26]:
#TAKES ~5 MINUTES TO RUN
games_end_indices = []
for x in range(len(games_end_on_play)):
    games_end_indices.append(games_end_on_play_df.where(games_end_on_play_df['GameID'] == games_end_on_play[x]).last_valid_index())

In [136]:
for x in games_end_indices:
    home_team = df.loc[x]['HomeTeam']
    away_team = df.loc[x]['AwayTeam']
    pos_team = df.loc[x]['posteam']
    if df.loc[x]['FieldGoalResult'] == 'Good':
        if pos_team == home_team:
            val = df.loc[x]['HomeScore'] + 3
            df.set_value(x, 'HomeScore', val)
        else:
            val = df.loc[x]['AwayScore'] + 3
            df.set_value(x, 'AwayScore', val)
    elif ('FUMBLES' in df.loc[x]['desc'] or 'INTERCEPTED' in df.loc[x]['desc'] or 'BLOCKED' in df.loc[x]['desc']) and 'TOUCHDOWN' in df.loc[x]['desc']:
        if pos_team == home_team:
            if df.loc[x]['ExPointResult'] == 'Made':
                val = df.loc[x]['AwayScore'] + 7
                df.set_value(x, 'AwayScore', val)
            elif df.loc[x]['TwoPointConv'] == 'Success':
                val = df.loc[x]['AwayScore'] + 8
                df.set_value(x, 'AwayScore', val)
            else:
                val = df.loc[x]['AwayScore'] + 6
                df.set_value(x, 'AwayScore', val)
        else:
            if df.loc[x]['ExPointResult'] == 'Made':
                val = df.loc[x]['HomeScore'] + 7
                df.set_value(x, 'HomeScore', val)
            elif df.loc[x]['TwoPointConv'] == 'Success':
                val = df.loc[x]['HomeScore'] + 8
                df.set_value(x, 'HomeScore', val)
            else:
                val = df.loc[x]['HomeScore'] + 6
                df.set_value(x, 'HomeScore', val)
    elif 'TOUCHDOWN' in df.loc[x]['desc']:
        if pos_team == away_team:
            if df.loc[x]['ExPointResult'] == 'Made':
                val = df.loc[x]['AwayScore'] + 7
                df.set_value(x, 'AwayScore', val)
            elif df.loc[x]['TwoPointConv'] == 'Success':
                val = df.loc[x]['AwayScore'] + 8
                df.set_value(x, 'AwayScore', val)
            else:
                val = df.loc[x]['AwayScore'] + 6
                df.set_value(x, 'AwayScore', val)
        else:
            if df.loc[x]['ExPointResult'] == 'Made':
                val = df.loc[x]['HomeScore'] + 7
                df.set_value(x, 'HomeScore', val)
            elif df.loc[x]['TwoPointConv'] == 'Success':
                val = df.loc[x]['HomeScore'] + 8
                df.set_value(x, 'HomeScore', val)
            else:
                val = df.loc[x]['HomeScore'] + 6
                df.set_value(x, 'HomeScore', val)
    elif 'SAFETY' in df.loc[x]['desc']:
        if pos_team == home_team:
            val = df.loc[x]['AwayScore'] + 2
            df.set_value(x, 'AwayScore', val)
        else:
            val = df.loc[x]['HomeScore'] + 2
            df.set_value(x, 'HomeScore', val)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[

In [145]:
df = df.drop(['Date', 'yrdln', 'time', 'TimeUnder', 
             'PlayTimeDiff', 'PlayAttempted', 'sp', 
              'Interceptor', 'Rusher', 'Rusher_ID', 'Receiver', 
              'Receiver_ID', 'Returner', 'BlockingPlayer', 'Tackler1', 
              'Tackler2', 'RecFumbPlayer', 'Challenge.Replay', 'ChalReplayResult', 
              'PenalizedPlayer', 'ScoreDiff', 'AbsScoreDiff', 'airEPA', 'yacEPA', 'Away_WP_pre', 
              'Away_WP_post', 'airWPA', 'yacWPA', 'Timeout_Indicator', 'Timeout_Team', 
              'posteam_timeouts_pre', 'HomeTimeouts_Remaining_Pre', 'AwayTimeouts_Remaining_Pre', 
              'HomeTimeouts_Remaining_Post', 'AwayTimeouts_Remaining_Post', 'Win_Prob', 
              'No_Score_Prob', 'Opp_Field_Goal_Prob', 'Opp_Safety_Prob', 'Opp_Touchdown_Prob', 
              'Field_Goal_Prob', 'Safety_Prob', 'Touchdown_Prob', 'ExPoint_Prob', 'TwoPoint_Prob'], axis = 1)

In [146]:
df.to_csv('cleaned_nfl_data.csv')